<a href="https://colab.research.google.com/github/aleksasekulicfon/Napredna-analiza-podataka/blob/main/Final_NAP_Projekat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Uvod i pozicioniranje eksperimenta

Prikazani segment koda predstavlja početnu pripremu eksperimenta čiji je cilj poređenje tri pristupa klasifikaciji teksta: (1) klasičnog machine learning pristupa, (2) zero-shot klasifikacije zasnovane na promptovanju LLM-a i (3) few-shot klasifikacije kao proširenja zero-shot pristupa uvođenjem malog broja primera u prompt.

 Predmet analize je sentiment u tvitovima na nivou entiteta, gde se sentiment određuje u odnosu na navedeni entitet, a ne samo na osnovu samog teksta poruke.

Korišćeni skup podataka je javno dostupan “Twitter Entity Sentiment Analysis” dataset preuzet sa Kaggle, pri čemu zapis sadrži entitet, tekst tvita i oznaku sentimenta.

#Instalacija biblioteka i formiranje radnog okruženja

Prvi korak je instalacija paketa koji pokrivaju sve faze rada: obradu podataka (pandas, numpy), klasično učenje i evaluaciju (scikit-learn), rad sa velikim jezičkim modelima (transformers, torch, accelerate), vizuelizacije (matplotlib, seaborn), kao i orkestraciju LLM poziva i standardizaciju promptovanja (langchain i langchain_groq). Ovakav izbor alata odgovara metodologiji koja eksplicitno predviđa paralelnu upotrebu klasičnog ML pristupa i LLM pristupa (zero-shot/few-shot).

Nakon instalacije sledi učitavanje (import) biblioteka. Time se obezbeđuje da su dostupni svi ključni objekti: TF-IDF vektorizator i klasifikatori za klasični ML pipeline, metrike za evaluaciju (accuracy, precision/recall/F1 i confusion matrix), kao i tokenizer i model iz Hugging Face ekosistema za LLM deo eksperimenta. Sama struktura import-a nagoveštava kasniji tok: najpre obrada i priprema teksta, zatim treniranje/validacija klasičnog modela, i na kraju LLM evaluacija kroz promptove.

#Detekcija hardverskog okruženja (CPU/GPU)

Deo koda koji proverava dostupnost CUDA podrške (torch.cuda.is_available()) služi da se automatski identifikuje da li je moguće izvršavanje na GPU-u. Iako se u ovom isečku još ne prebacuju modeli na uređaj, ovakva provera je standardna priprema za kasniju fazu, gde se inference ili obrada sa modelima može značajno ubrzati korišćenjem GPU-a.

#Učitavanje skupa podataka i definisanje strukture zapisa

Kôd zatim definiše očekivanu strukturu zapisa kroz četiri kolone: ID zapisa/tvita, entitet, oznaku klase/sentimenta i tekst tvita.

 Nakon toga se učitavaju dve datoteke koje čine osnovu eksperimenta: twitter_training.csv kao trening skup i twitter_validation.csv kao validacioni skup.

 U dokumentu je naglašeno i da su u originalnom dataset-u prisutne oznake Positive, Negative, Neutral i Irrelevant, uz napomenu da se Irrelevant može tretirati kao Neutral kako bi se problem sveo na tri klase.

U samom kodu se dodatno uklanjaju redovi bez tekstualnog sadržaja (dropna(subset=['text'])), čime se obezbeđuje da dalji koraci (vektorizacija, treniranje i promptovanje) rade nad validnim tekstualnim ulazima.

#Konstrukcija tekstualnog ulaza koji čuva “entity-level” definiciju

Ključni korak u pripremi podataka je formiranje nove kolone input_text, gde se entitet i tekst tvita konkateniraju u jedinstven string oblika:

entity: <ENTITET> | tweet: <TEKST>

Ovaj format eksplicitno uvodi entitet kao deo tekstualnog konteksta, čime se zadržava definicija zadatka (sentiment “o entitetu”), dok ulaz ostaje isključivo tekstualnog tipa.

 Takva formulacija je posebno korisna jer omogućava da i klasični ML modeli (koji očekuju tekst) i LLM pristupi (koji rade nad promptom) dobiju ujednačen, porediv ulaz, bez gubitka informacije o tome prema kome je sentiment usmeren.

#Brza provera ispravnosti pripreme

Na kraju se ispisuje jedan primer iz input_text kolone. Ovaj ispis ima ulogu “sanity check”-a: potvrđuje da su CSV fajlovi pravilno učitani, da su kolone korektno mapirane i da je format ulaza uspešno formiran pre ulaska u naredne faze eksperimenta (vektorizacija, treniranje i LLM evaluacija).

In [ ]:
# 1. Instalacija i Učitavanje
!pip install pandas scikit-learn transformers accelerate torch matplotlib seaborn langchain langchain_groq -q

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from sklearn.pipeline import Pipeline
import re
import json

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Podešen uređaj: {device}")

columns = ['id', 'entity', 'sentiment', 'text']
train_df = pd.read_csv('twitter_training.csv', names=columns, header=None).dropna(subset=['text'])
val_df = pd.read_csv('twitter_validation.csv', names=columns, header=None).dropna(subset=['text'])

train_df['input_text'] = "entity: " + train_df['entity'] + " | tweet: " + train_df['text']
val_df['input_text'] = "entity: " + val_df['entity'] + " | tweet: " + val_df['text']

print("Primer ulaznog podatka:", train_df['input_text'].iloc[0])

# 2. Klasični Machine Learning (ML)

#Priprema teksta i implementacija klasičnog ML pristupa

Nakon inicijalnog učitavanja skupa podataka, sledeći korak je priprema tekstualnog sadržaja za klasičnu obradu i treniranje modela. U tu svrhu uvodi se jednostavna funkcija za čišćenje teksta, čiji je cilj da ukloni elemente koji često predstavljaju šum u tvitovima (linkove, oznake korisnika i razne simbole), a da istovremeno sačuva deo izražajnih signala poput uzvičnika i upitnika.

Funkcija smart_clean_text najpre osigurava da je ulaz uvek string, zatim uklanja URL-ove (npr. http...), briše “mention” oznake korisnika (npr. @username), a potom odstranjuje sve karaktere koji nisu slova, razmaci ili osnovni interpunkcijski znaci ! i ?. Na kraju se tekst prevodi u mala slova. Ova transformacija ima dve glavne posledice: (1) smanjuje se broj različitih tokena (npr. “Game” i “game” postaju isto), i (2) model dobija “čistiji” ulaz gde dominiraju reči i osnovna emotivna interpunkcija, što može poboljšati stabilnost TF-IDF reprezentacije.

Dobijeni čisti tekst se čuva u novim kolonama clean_text i za trening i za validacioni skup, čime se obezbeđuje da oba skupa prolaze kroz identičnu pripremu. Nakon toga se ponovo formira kolona input_text u standardizovanom formatu “entity: … | tweet: …”, ali sada koristeći očišćenu verziju tvita. Time se dodatno osigurava da se entitet ne izgubi i da ulaz ostane usmeren na entity-level sentiment.

#Konsolidacija klasa: tretiranje “Irrelevant” kao “Neutral”

Pre treniranja modela, oznaka sentimenta “Irrelevant” se mapira na “Neutral”. Ovaj korak ima praktičnu svrhu: umesto da se model bavi dodatnom klasom koja često predstavlja “neodnos prema entitetu”, problem se svodi na tri standardne klase (Positive, Negative, Neutral). Na taj način evaluacija postaje preglednija, a klasifikator se fokusira na distinkciju koja je obično najznačajnija u sentiment analizi.

#Pipeline: TF-IDF + Linearni SVM kao jaka bazna linija

Centralni deo pristupa je izgradnja Pipeline objekta koji spaja dve faze u jedan konzistentan tok obrade:

TF-IDF vektorizacija
Tekst se pretvara u numeričku reprezentaciju pomoću TF-IDF pristupa. Podešavanja su izabrana tako da uhvate i pojedinačne reči i kratke fraze:

ngram_range=(1,3) omogućava unigrame, bigrame i trigrame, što često pomaže u sentimentu jer se negativni/pozitivni ton neretko izražava kroz fraze (“not good”, “very bad”, “so happy”).

max_features=25000 ograničava dimenzionalnost na najrelevantnije karakteristike kako bi model ostao efikasan.

min_df=2 izbacuje termine koji se pojavljuju veoma retko, jer su često šum i slabo generalizuju.

max_df=0.9 uklanja termine koji se pojavljuju u gotovo svim dokumentima (previše opšti termini), jer ne nose diskriminativnu informaciju.

LinearSVC (linearni SVM klasifikator)
Na TF-IDF vektorima trenira se linearni SVM. Linearni SVM je česta i vrlo jaka bazna metoda za klasifikaciju teksta, naročito kada se koristi TF-IDF sa n-gramima. Parametri poput max_iter=5000 obezbeđuju dovoljno iteracija za konvergenciju, dok random_state=42 obezbeđuje reproduktivnost.

#Optimizacija hiperparametara: Grid Search sa unakrsnom validacijom

Da bi se dobila najbolja moguća varijanta SVM modela, koristi se GridSearchCV. Umesto ručnog pogađanja parametara, pretražuje se mreža kombinacija:

clf__C: jačina regularizacije (0.1, 1, 10). Manje vrednosti više “peglaju” model (više regularizacije), veće vrednosti dopuštaju složenije granice odluke.

clf__class_weight: ili bez težina, ili balanced. Opcija balanced je korisna kada klase nisu jednako zastupljene, jer automatski pojačava važnost ređih klasa.

tfidf__sublinear_tf: uključivanje sublinear TF skaliranja (True/False), što može pomoći jer ublažava efekat ekstremno čestih reči u pojedinačnom dokumentu.

Unakrsna validacija cv=3 deli trening skup na tri dela i rotira treniranje/validaciju, čime se smanjuje rizik da izabrani parametri budu “srećno pogođeni” na jednoj podeli. n_jobs=-1 koristi sve dostupne CPU jezgre radi bržeg pretraživanja, a verbose=1 služi da se proces prati tokom izvršavanja.

Kada se završi pretraga, ispisuju se najbolji parametri i uzima se najbolji model (best_estimator_) za finalnu evaluaciju na validacionom skupu.

#Evaluacija: tačnost, detaljan izveštaj i matrica konfuzije

Najbolji model se zatim koristi da predvidi sentiment za val_df['input_text']. Evaluacija se radi na nekoliko nivoa:

Accuracy daje ukupni procenat tačno klasifikovanih primera.

classification_report prikazuje preciznost, odziv i F1 meru po klasama, što je važno jer model može imati dobru ukupnu tačnost, a da pritom loše radi na jednoj klasi (npr. Neutral).

confusion_matrix daje uvid u tipične greške: npr. da li model često meša Neutral sa Negative, ili Positive sa Neutral.

Vizuelizacija matrice konfuzije kroz heatmap prikaz dodatno olakšava interpretaciju, jer se odmah vidi gde su najveća “curenja” između klasa. Oznake su svedene na kratke forme (Pos/Neg/Neu) radi čitljivosti, a naslov jasno navodi da se radi o SVM modelu.

In [ ]:
from sklearn.svm import LinearSVC

def smart_clean_text(text):
    text = str(text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s!?]', '', text)
    text = text.lower()
    return text

train_df['clean_text'] = train_df['text'].apply(smart_clean_text)
val_df['clean_text'] = val_df['text'].apply(smart_clean_text)

train_df['input_text'] = "entity: " + train_df['entity'] + " | tweet: " + train_df['clean_text']
val_df['input_text'] = "entity: " + val_df['entity'] + " | tweet: " + val_df['clean_text']

train_df['sentiment'] = train_df['sentiment'].replace('Irrelevant', 'Neutral')
val_df['sentiment'] = val_df['sentiment'].replace('Irrelevant', 'Neutral')

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=25000,
        ngram_range=(1, 3),
        min_df=2,
        max_df=0.9
    )),
    ('clf', LinearSVC(
        random_state=42,
        dual='auto',
        max_iter=5000
    ))
])

param_grid = {
    'clf__C': [0.1, 1, 10],
    'clf__class_weight': [None, 'balanced'],
    'tfidf__sublinear_tf': [True, False]
}

print("\nPokrećem optimizaciju SVM modela (Grid Search)...")
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train_df['input_text'], train_df['sentiment'])

print(f"Najbolji parametri: {grid_search.best_params_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(val_df['input_text'])
y_val = val_df['sentiment']

acc = accuracy_score(y_val, y_pred)
print(f"\n--- MAKSIMALNA TAČNOST SA SVM: {acc:.4f} ---")
print(classification_report(y_val, y_pred))

plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_val, y_pred, labels=['Positive', 'Negative', 'Neutral'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Pos', 'Neg', 'Neu'],
            yticklabels=['Pos', 'Neg', 'Neu'])
plt.title('Confusion Matrix (SVM Model)')
plt.show()

#Učitavanje i pozivanje LLM-a preko Groq servisa

Ovaj segment koda uvodi treći deo eksperimenta: rad sa velikim jezičkim modelom (LLM) bez treniranja, kroz pozive eksternom servisu. Ideja je da se umesto klasičnog “fit” procesa koristi inference (odgovaranje modela na prompt), čime se omogućava zero-shot ili few-shot klasifikacija sentimenta. Da bi taj pristup bio stabilan u praksi, kod obuhvata tri ključne celine: (1) bezbedno preuzimanje API ključa, (2) inicijalizaciju LLM klijenta sa kontrolisanim parametrima i (3) robusnu funkciju za slanje prompta sa keširanjem i retry mehanizmom.

#1) Log poruka i importi za mrežni poziv

Na početku se štampa poruka koja služi kao marker u notebook-u da započinje sekcija učitavanja LLM-a. Zatim se uvoze biblioteke:

os za čitanje i postavljanje promenljivih okruženja (API ključ),

random i sleep za uvođenje nasumične pauze između ponovnih pokušaja,

ChatGroq iz langchain_groq kao klijent koji pojednostavljuje pozive Groq API-ju.

Ovo je tipična priprema za mrežno pozivanje LLM-a gde se očekuju povremeni prekidi ili rate-limit situacije.

#2) Bezbedno učitavanje GROQ_API_KEY ključa

Funkcija get_groq_key() rešava čest problem: isti kod treba da radi i lokalno i u Google Colab okruženju.

Mehanizam je postavljen hijerarhijski:

Prvo se proverava sistemsko okruženje (os.getenv("GROQ_API_KEY")).
Ako postoji, odmah se vraća ključ.

Ako ne postoji, pokušava se učitavanje iz Colab userdata (kroz google.colab.userdata).
Ako se ključ pronađe, dodatno se setuje i u os.environ["GROQ_API_KEY"], što je praktično jer sve kasnije biblioteke koje očekuju env var sada mogu da je vide.

Ako oba izvora zakažu, baca se RuntimeError, čime se eksplicitno prekida izvršavanje i jasno signalizira da bez ključa nema daljeg rada.

Poziv get_groq_key() odmah nakon definicije funkcije je nameran: cilj je da se greška detektuje što ranije, pre nego što se napravi klijent ili krene sa pozivima modela.

#3) Izbor modela i inicijalizacija ChatGroq klijenta

Zatim se definiše:

GROQ_MODEL = "llama-3.1-8b-instant"


što znači da će se koristiti konkretna varijanta Llama modela, optimizovana za brze odgovore.

Nakon toga se pravi llm = ChatGroq(...) sa važnim parametrima:

temperature=0.0: model postaje maksimalno determinističan.
To je posebno bitno za klasifikaciju sentimenta, jer se očekuje konzistentan izlaz (npr. uvek “Positive/Negative/Neutral”), a ne kreativne varijacije.

max_tokens=6: ograničava dužinu odgovora na veoma kratku formu.
Za klasifikaciju je često dovoljno 1–2 reči (“Positive”, “Negative”, “Neutral”), pa se ovim smanjuje rizik da model generiše objašnjenja, dodatne rečenice ili “priču” umesto etikete.

Ovaj deo praktično “zakucava” LLM da se ponaša kao klasifikator, a ne kao chat asistent.

#4) Keširanje odgovora radi brzine i stabilnosti

Definiše se _cache: dict[str, str] = {} kao rečnik koji mapira prompt → odgovor.

Keš ima dve funkcije:

Ubrzanje: ako se isti prompt ponovo pošalje (što se često dešava u evaluaciji ili pri grešci), rezultat se odmah vraća bez novog API poziva.

Stabilnost i trošak: smanjuje broj zahteva prema servisu, što smanjuje šansu za rate limit i ubrzava workflow.

#5) Funkcija query_llm: robustan poziv sa retry + exponential backoff

Funkcija query_llm(...) je centralni deo celog segmenta. Ona pretvara poziv modelu u “siguran” API:

##5.1 Validacija ulaza

Najpre se prompt “trimuje” i proverava da nije prazan. Ako jeste, vraća se prazan string. Ovo sprečava besmislene pozive.

##5.2 Keš pre mreže

Ako je use_cache=True i prompt postoji u _cache, vraća se keširana vrednost. To je najbrži put.

##5.3 Retry petlja

Ako nije u kešu, funkcija pokušava da pozove:

text = (llm.invoke(prompt).content or "").strip()


i vrati sadržaj odgovora.

Ako dođe do izuzetka (mrežni prekid, rate limit, timeout, backend error), hvata se greška i prelazi se na sledeći pokušaj sve do retries.

##5.4 Exponential backoff + jitter

Između pokušaja se pravi pauza:

osnovni delay raste eksponencijalno (base_delay * 2^(attempt-1)),

“seče” se na max_delay da pauze ne postanu preduge,

dodaje se mali slučajni dodatak (random.uniform(0, 0.25)) da se izbegne situacija da više procesa “udari” u isti trenutak (tzv. jitter).

Ovaj pattern je standard u mrežnim sistemima jer značajno povećava šanse da sledeći pokušaj uspe.

##5.5 Kontrolisani fallback

Ako ni posle svih pokušaja poziv ne uspe, štampa se upozorenje i vraća prazan string. Time se omogućava da ostatak evaluacije nastavi (npr. da se preskoči taj primer), umesto da ceo proces pukne.

In [ ]:
print("--- Učitavanje LLM Modela (Groq) ---")

import os
import random
from time import sleep
from langchain_groq import ChatGroq


def get_groq_key() -> str:
    key = os.getenv("GROQ_API_KEY")
    if key:
        return key
    try:
        from google.colab import userdata
        key = userdata.get("GROQ_API_KEY")
        if key:
            os.environ["GROQ_API_KEY"] = key
            return key
    except Exception:
        pass

    raise RuntimeError("GROQ_API_KEY nije pronađen (ni ENV ni Colab userdata).")


get_groq_key()

GROQ_MODEL = "llama-3.1-8b-instant"

llm = ChatGroq(
    model=GROQ_MODEL,
    temperature=0.0,
    max_tokens=6,
)

_cache: dict[str, str] = {}


def query_llm(
    prompt: str,
    retries: int = 5,
    base_delay: float = 0.7,
    max_delay: float = 8.0,
    use_cache: bool = True,
) -> str:
    prompt = (prompt or "").strip()
    if not prompt:
        return ""

    if use_cache and prompt in _cache:
        return _cache[prompt]

    last_err = None
    for attempt in range(1, retries + 1):
        try:
            text = (llm.invoke(prompt).content or "").strip()
            if use_cache:
                _cache[prompt] = text
            return text
        except Exception as e:
            last_err = e
            if attempt == retries:
                break
            delay = min(max_delay, base_delay * (2 ** (attempt - 1))) + random.uniform(0, 0.25)
            sleep(delay)

    print(f"[WARN] Groq poziv nije uspeo nakon {retries} pokušaja: {last_err}")
    return ""

print(f"Model učitan! Groq model: {GROQ_MODEL}")

#Zero-shot klasifikacija sentimenta na nivou entiteta (3 klase)

Ovaj segment koda predstavlja implementaciju zero-shot pristupa za klasifikaciju sentimenta, gde se veliki jezički model koristi kao “gotov” klasifikator bez ikakvog dodatnog treniranja na konkretnom skupu podataka. Ideja zero-shot strategije je da se modelu kroz prompt precizno objasni zadatak i striktno definišu dozvoljene izlazne klase, a zatim se od njega traži da za svaki primer vrati etiketu sentimenta. U ovom slučaju problem je dodatno pojednostavljen na tri klase (Positive, Negative, Neutral) tako što se “Irrelevant” preslikava na “Neutral”, čime se dobija konzistentna podela pogodna za poređenje sa klasičnim ML modelom.

#Konsolidacija oznaka i definisanje “ground truth” vrednosti

Na početku se u validacionom skupu ponovo vrši mapiranje klase “Irrelevant” na “Neutral”. Time se obezbeđuje da se evaluacija radi u istom režimu od tri klase, bez četvrte kategorije koja često predstavlja “nije o ovom entitetu”. Nakon toga se formira y_val, tj. niz stvarnih oznaka (ground truth) iz validacionog skupa, koji će se koristiti kasnije za poređenje sa predikcijama LLM-a.

#Kreiranje zero-shot prompta: precizna instrukcija i kontrola izlaza

Funkcija create_zero_shot_prompt(text, entity) konstruiše prompt koji ima nekoliko pažljivo osmišljenih elemenata:

Uloga modela: prompt odmah postavlja model u ulogu “entity-level tweet sentiment classifier”, što je važno jer modelu daje jasan kontekst i očekivani tip odgovora.

Fokus na entitet: eksplicitno se kaže da se sentiment klasifikuje “towards the entity” i navodi se konkretan entitet pod navodnicima. Ovo sprečava tipičnu grešku gde bi model klasifikovao opšti sentiment tvita, a ne odnos prema entitetu.

Striktnost izlaza: prompt traži da se vrati tačno jedna od tri etikete. Ovim se modelu jasno zabranjuju objašnjenja, dodatne reči i nijansirane formulacije (“mostly positive”, “somewhat negative” i sl.).

Definicija Neutral klase: pošto je “Neutral” najšira i često najproblematičnija kategorija, prompt sadrži jasna pravila kada se bira Neutral:

ako je tvit nerelevantan za entitet,

ako nema jasnog sentimenta,

ako je informativna objava/vest,

ako je pitanje bez izraženog stava.

Ovo je posebno važno jer u tvitovima ima mnogo sadržaja koji nije emocionalno obojen, pa bez ovih pravila model može prečesto “halucinirati” pozitivan ili negativan ton.

Na kraju prompt ubacuje konkretan tvit i ostavlja format “Label:” kao signal da je sledeće što treba da dođe samo jedna etiketa.

#Inferencija nad validacionim skupom: iteracija i prikupljanje predikcija

Nakon definicije prompta, pokreće se petlja kroz sve redove val_df. Koristi se tqdm kako bi se dobio progress bar i jasna povratna informacija o toku inferencije, što je bitno jer LLM pozivi po pravilu traju duže od klasične predikcije.

Za svaki red:

uzima se originalni tekst tvita (row['text']) i entitet (row['entity']),

generiše se prompt,

šalje se modelu preko query_llm(prompt),

rezultat (odgovor modela) se dodaje u listu y_pred_zero_shot.

Ovaj deo koda u praksi predstavlja “batch” evaluaciju, samo što se radi sekvencijalno (primer po primer), jer se pozivi vrše prema udaljenom LLM servisu.

#Normalizacija izlaza: čišćenje nepredvidivih odgovora modela

Iako prompt traži striktno jednu od tri etikete, LLM ponekad može vratiti:

etiketu sa dodatnim rečima (“Positive sentiment”, “Label: Negative”),

razlike u velikim/malim slovima,

ili čak kratak komentar.

Zbog toga se uvodi funkcija clean_prediction(pred) koja radi jednostavnu normalizaciju:

sve spušta na mala slova,

proverava da li se u izlazu pojavljuje reč “positive” ili “negative”,

a ako ne prepozna ni jedno, podrazumeva “Neutral”.

Ovaj “fallback” na Neutral je pragmatičan: bolje je da nejasan ili pogrešno formatiran odgovor bude tretiran kao Neutral nego da proizvede slučajnu grešku ili pogrešnu klasu koja bi narušila evaluaciju.

Nakon toga se formira y_pred_zero_shot_clean, odnosno lista finalnih predikcija spremnih za poređenje sa y_val.

#Evaluacija zero-shot pristupa: izveštaj i tačnost

Na kraju se štampaju rezultati kroz dve ključne metrike:

classification_report: daje preciznost, odziv i F1 meru za svaku klasu (Positive, Negative, Neutral). Ovo je važno jer accuracy sama po sebi može biti varljiva, naročito ako je klasa Neutral dominantna.

accuracy_score: daje ukupnu tačnost kao jednu brojčanu vrednost, korisnu za brzo poređenje sa SVM baseline-om i sa few-shot pristupom.

In [ ]:
print("--- 2. ZERO-SHOT PRISTUP (3 KLASE) ---")

val_df['sentiment'] = val_df['sentiment'].replace('Irrelevant', 'Neutral')
y_val = val_df['sentiment']

def create_zero_shot_prompt(text, entity):
    return f"""You are an entity-level tweet sentiment classifier.

Classify the sentiment of the tweet towards the entity "{entity}".

Return EXACTLY one of these labels:
Positive
Negative
Neutral

Rules for Neutral:
- If the tweet is irrelevant to the entity, return Neutral.
- If there is no clear sentiment, return Neutral.
- If the tweet is news/update/announcement (informative), return Neutral.
- If the tweet is a question asking for info without an opinion, return Neutral.

Tweet: {text}
Label:"""

y_pred_zero_shot = []
for _, row in tqdm(val_df.iterrows(), total=val_df.shape[0], desc="Zero-shot inference"):
    prompt = create_zero_shot_prompt(row['text'], row['entity'])
    response = query_llm(prompt)
    y_pred_zero_shot.append(response)

def clean_prediction(pred):
    pred = str(pred).lower()
    if "positive" in pred:
        return "Positive"
    if "negative" in pred:
        return "Negative"
    return "Neutral"

y_pred_zero_shot_clean = [clean_prediction(p) for p in y_pred_zero_shot]

print("\nRezultati Zero-shot pristupa (3 KLASE):")
print(classification_report(y_val, y_pred_zero_shot_clean, labels=["Positive","Negative","Neutral"]))
print("Accuracy:", accuracy_score(y_val, y_pred_zero_shot_clean))

In [ ]:
labels = ["Positive", "Negative", "Neutral"]

cm = confusion_matrix(y_val, y_pred_zero_shot_clean, labels=labels)

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix — Zero-shot (3 klase)")
plt.xlabel("Predviđeno")
plt.ylabel("Stvarno")
plt.show()

# **5. Few-shot Klasifikacija**

Ovde biramo nekoliko primera iz trening seta i ubacujemo ih u prompt kako bismo modelu "pokazali" šta želimo.

#Few-shot pristup: “kalibrisanje” LLM-a primerima i striktno JSON formatiranje izlaza

Ovaj segment koda predstavlja few-shot varijantu LLM klasifikacije sentimenta, gde se modelu ne daje samo instrukcija (kao u zero-shot režimu), već i mali skup reprezentativnih primera koji ilustruju kako treba da izgleda ulaz i kakav izlaz se očekuje. Cilj few-shot strategije je da se model “usmeri” na tačan zadatak (entity-level sentiment) i da se smanji varijabilnost u odgovoru, naročito u graničnim slučajevima (vesti, pitanja, nerelevantni tvitovi). Pored toga, ovde je poseban akcenat na robustnosti: izlaz se forsira u obliku JSON-a, a predikcije se parsiraju defensivno, uz fallback pravila.

#1) Izbor few-shot primera: balans po klasama

Na početku se definiše lista few_shot_examples koja sadrži ukupno šest primera, po dva za svaku klasu:

Positive primeri su kratki, entuzijastični (“heck yeah”, “looking forward…”).

Negative primeri imaju jasno izraženo nezadovoljstvo (“worst customer service”, “really sucks”).

Neutral primeri su tipične vesti/objave (“Leak: specifications…”, “Blizzard confirms…”).

Ovaj izbor nije slučajan: few-shot primeri rade kao “sidra” za model — daju mu referencu šta se smatra pozitivnim, negativnim i neutralnim u specifičnom domenu tvitova, i naročito pomažu da “Neutral” ne bude pomešan sa slabim pozitivnim/negativnim tonom.

#2) Definisanje pomoćnih pravila i regularnih izraza

Zatim se definišu konstante i regex obrasci:

LABELS = ["Positive", "Negative", "Neutral"] predstavlja jedini dozvoljeni skup etiketa.

_URL_RE služi za uklanjanje URL-ova (jer linkovi često ne doprinose sentimentu, a mogu “zbuniti” prompt).

_JSON_RE pokušava da pronađe JSON objekat unutar odgovora modela, čak i ako model “prekrši” pravilo i doda dodatni tekst.

_LABEL_RE je rezervni mehanizam: ako JSON parsiranje ne uspe, pokušava se barem izvući jedna od reči “positive/negative/neutral” iz odgovora.

Ova kombinacija predstavlja tipičan “defensive parsing” pristup: kod je pripremljen za realnost da LLM ponekad ne prati format do kraja.

#3) Gradnja few-shot konteksta: primeri kao mini-dataset u promptu

Funkcija _strip_urls čisti URL-ove iz teksta primera, a zatim se few_shot_context formira kao blok koji spaja sve primere u isti format:

Entity: ...

Tweet: ...

JSON: {"label":"..."}

Važno je što primeri uključuju i ulaz i izlaz u tačno onom obliku koji se kasnije traži od modela. Time se modelu implicitno pokazuje: “Evo kako izgleda ispravan odgovor”.

Ispis “preview” dela konteksta nije neophodan za logiku klasifikacije, ali je praktičan za debug: može se vizuelno proveriti da li su primeri formatirani ispravno i da li se slučajno ne ubacuju URL-ovi ili ružni karakteri.

#4) Prompt za few-shot: striktno JSON pravilo i neutral definicija

Funkcija create_few_shot_prompt(text, entity) konstruiše prompt koji je strožiji nego u zero-shot varijanti:

Model dobija ulogu klasifikatora.

Eksplicitno se traži da vrati samo validan JSON objekat u jednom od tri oblika:

{"label":"Positive"}

{"label":"Negative"}

{"label":"Neutral"}

Zabranjuje se markdown, backticks, komentari i bilo kakav dodatni tekst.

Zatim se ponavlja definicija Neutral klase kroz jasna pravila (nerelevantno, nema stava, vesti, pitanje bez mišljenja), a ispod toga se dodaje blok Examples: koji sadrži prethodno izgrađen few_shot_context.

Na kraju prompt prelazi na konkretan primer (“Now classify:”), navodi entitet i tvit iz validacionog skupa, i završava sa “JSON:” kao signal da se očekuje direktno vraćanje JSON-a.

Ovaj dizajn prompta pokušava da reši dva tipična problema LLM evaluacije:

model koji objašnjava umesto da klasifikuje,

model koji vraća labelu u slobodnom tekstu, a ne u tačno traženom formatu.

#5) Parsiranje odgovora: robustan sistem sa više slojeva

Funkcija parse_prediction(response) pretvara sirov odgovor LLM-a u jednu od tri klase i radi to u više koraka:

Ako je odgovor prazan, automatski se vraća “Neutral” (bezbedan fallback).

Prvo se traži JSON segment pomoću _JSON_RE. Ako se pronađe, pokušava se json.loads, zatim se uzima label, normalizuje (title-case) i proverava da li je u LABELS.

Ako JSON parsiranje ne uspe, pretražuje se odgovor da li sadrži reč “positive/negative/neutral” (regex _LABEL_RE).

Ako ni to ne uspe, vraća se “Neutral”.

Ovaj pristup obezbeđuje da evaluacija ne “pukne” zbog jednog loše formatiranog odgovora, i da rezultat uvek bude validna klasa. U eksperimentima sa LLM-ovima ovo je kritično, jer i mali procenat nepravilno formatiranih odgovora može pokvariti evaluaciju ako se ne tretira.

#6) Inference petlja sa dodatnim keširanjem na nivou primera

Za razliku od ranijeg query_llm keša (koji kešira po promptu), ovde se uvodi još jedan sloj keširanja:

pred_cache čuva već izračunatu predikciju za par (entity, text).

raw_cache čuva sirov odgovor modela za isti ključ (korisno za kasniju analizu grešaka i debug).

Petlja prolazi kroz validacioni skup preko itertuples, što je efikasnije od iterrows. Za svaki (entity, text):

formira se ključ (entity, text),

ako je već viđen, predikcija se odmah vraća iz pred_cache,

u suprotnom se pravi prompt, šalje modelu, parsira odgovor i upisuje u cache.

Ovo je praktično u scenarijima gde se u dataset-u pojavljuju duplikati ili kada se eksperimenti ponavljaju: značajno se smanjuje broj LLM poziva, vreme i trošak.

#7) Evaluacija i metrika “cache miss” kao indikator efikasnosti

Na kraju se ispisuje:

classification_report za tri klase, uz zero_division=0 da se izbegnu upozorenja u slučaju da model nijednom ne predvidi neku klasu.

accuracy_score kao jednostavna ukupna metrika.

“Unique prompts (cache miss)” kao odnos jedinstvenih primera prema ukupnom broju redova — indikator koliko je keš pomogao (tj. koliko duplikata ili ponovljenih upita postoji).

In [ ]:
few_shot_examples = [
    # Positive
    {"entity": "Xbox(Xseries)", "text": "got a new xbox series x from best buy. heck yeah", "label": "Positive"},
    {"entity": "Borderlands", "text": "Going to finish up volume 2 today. I've got some awesome serv... looking forward to a good stream!", "label": "Positive"},

    # Negative
    {"entity": "Amazon", "text": "@amazon probably some of the worst customer service I’ve had to deal with lately", "label": "Negative"},
    {"entity": "Borderlands", "text": "Man Gearbox really needs to fix this dissapointing drops... Really sucks alot", "label": "Negative"},

    # Neutral
    {"entity": "Xbox(Xseries)", "text": "(Leak) Final specifications for PS5 and Xbox Series X: Two Monsters!", "label": "Neutral"},
    {"entity": "WorldOfCraft", "text": "Blizzard confirms that World of Warcraft: Shadowlands isn't c... overclock3d.net/news/software/…", "label": "Neutral"},
]

LABELS = ["Positive", "Negative", "Neutral"]
_URL_RE = re.compile(r"http\S+|www\.\S+", re.IGNORECASE)
_JSON_RE = re.compile(r"\{.*?\}", re.DOTALL)
_LABEL_RE = re.compile(r"\b(positive|negative|neutral)\b", re.IGNORECASE)

def _strip_urls(s: str) -> str:
    return _URL_RE.sub("", str(s)).strip()

few_shot_context = "\n\n".join(
    f'Entity: {ex["entity"]}\n'
    f'Tweet: {_strip_urls(ex["text"])}\n'
    f'JSON: {{"label":"{ex["label"]}"}}'
    for ex in few_shot_examples
)

print("Few-shot context preview:\n", few_shot_context[:1200], "...\n")

def create_few_shot_prompt(text, entity):
    return f"""You are an entity-level tweet sentiment classifier.

Return ONLY a valid JSON object:
{{"label":"Positive"}} OR {{"label":"Negative"}} OR {{"label":"Neutral"}}
Do not return markdown, backticks, comments, or any extra text.

Neutral means:
- irrelevant to entity
- no clear sentiment
- factual news/update
- question without opinion

Examples:
{few_shot_context}

Now classify:
Entity: {entity}
Tweet: {text}

JSON:
""".strip()

def parse_prediction(response: str) -> str:
    txt = str(response).strip()
    if not txt:
        return "Neutral"

    m = _JSON_RE.search(txt)
    if m:
        try:
            payload = json.loads(m.group(0))
            lbl = str(payload.get("label", "")).strip().title()
            if lbl in LABELS:
                return lbl
        except Exception:
            pass

    m = _LABEL_RE.search(txt)
    if m:
        return m.group(1).title()

    return "Neutral"

y_val = val_df["sentiment"].replace("Irrelevant", "Neutral").tolist()
y_pred_few_shot = []

pred_cache = {}
raw_cache = {}

for entity, text in tqdm(
    val_df[["entity", "text"]].itertuples(index=False, name=None),
    total=len(val_df),
    desc="Few-shot (Groq JSON)"
):
    key = (entity, text)
    if key in pred_cache:
        y_pred_few_shot.append(pred_cache[key])
        continue

    prompt = create_few_shot_prompt(text, entity)
    response = query_llm(prompt)
    pred = parse_prediction(response)

    pred_cache[key] = pred
    raw_cache[key] = response
    y_pred_few_shot.append(pred)

print(classification_report(y_val, y_pred_few_shot, labels=LABELS, zero_division=0))
print("Accuracy:", accuracy_score(y_val, y_pred_few_shot))
print(f"Unique prompts (cache miss): {len(pred_cache)} / {len(val_df)}")

In [ ]:
labels = ["Positive", "Negative", "Neutral"]

cm = confusion_matrix(y_val, y_pred_few_shot, labels=labels)

plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix — Few-shot (3 klase)")
plt.xlabel("Predviđeno")
plt.ylabel("Stvarno")
plt.show()

#**6. Poredjenje rezultata (Vizuelizacija i Tabela)**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

print("--- 6. FINALNA VIZUELIZACIJA I POREĐENJE ---")

acc_ml = accuracy_score(y_val, y_pred)
acc_zero = accuracy_score(y_val, y_pred_zero_shot_clean)
acc_few = accuracy_score(y_val, y_pred_few_shot)

results_df = pd.DataFrame({
    'Metod': ['Classical ML (SVM)', 'Zero-shot (LLM)', 'Few-shot (LLM)'],
    'Accuracy': [acc_ml, acc_zero, acc_few]
})

print("\nTABELA REZULTATA:")
print(results_df)

plt.figure(figsize=(10, 6))
sns.barplot(x='Metod', y='Accuracy', data=results_df, palette='viridis', hue='Metod', legend=False)

plt.title('Finalno poređenje tačnosti (Accuracy)', fontsize=15)
plt.ylim(0, 1.1)
plt.ylabel('Tačnost', fontsize=12)
plt.xlabel('Pristup', fontsize=12)

for index, row in results_df.iterrows():
    plt.text(index, row.Accuracy + 0.02, f"{row.Accuracy*100:.1f}%",
             color='black', ha="center", fontweight='bold', fontsize=12)
plt.show()

print(f"\nDetaljna analiza grešaka za najbolji model (SVM):")
labels = ['Positive', 'Negative', 'Neutral']

cm = confusion_matrix(y_val, y_pred, labels=labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels, annot_kws={"size": 14})
plt.title('Confusion Matrix: SVM (Pobednik)', fontsize=15)
plt.ylabel('Stvarna klasa', fontsize=12)
plt.xlabel('Predviđena klasa', fontsize=12)
plt.show()

#Finalna vizuelizacija i poređenje pristupa

Završni segment koda služi da objedini rezultate sva tri pristupa i da ih prikaže na način koji je odmah razumljiv: prvo kao tabelu i bar-čart (poređenje tačnosti), a zatim kao konfuzionu matricu za najbolji model, kako bi se videlo gde i kako model greši. Ovaj deo je tipično “finalni slajd” u analizi: sumira performanse, potvrđuje pobednika i daje smernice za tumačenje razlika između metoda.

#1) Ugradnja stvarnih rezultata i logika poređenja

Pre samog plotovanja izračunavaju se tačnosti:

acc_ml za klasični ML pristup (TF-IDF + linearni SVM),

acc_zero za zero-shot LLM predikcije,

acc_few za few-shot LLM predikcije.

U konkretnom eksperimentu dobijeni su sledeći rezultati:

Classical ML (SVM): 92,2% accuracy

Zero-shot (LLM): 64% accuracy

Few-shot (LLM): 62% accuracy

Ovi brojevi se zatim smeštaju u DataFrame results_df sa dve kolone: naziv metode i accuracy. Time se dobija uredna struktura pogodna i za štampu (tabela) i za vizuelizaciju.

#2) Tabela rezultata: brzi pregled pre grafika

Štampanje results_df ima praktičnu ulogu: pre grafičkog prikaza postoji “čist” numerički rezime koji se lako može preneti u izveštaj ili prezentaciju. Tabela jasno pokazuje drastičnu razliku između treniranog modela i LLM pristupa bez treniranja.

#3) Bar-čart: vizuelno poređenje tačnosti

Zatim se crta bar-čart gde je:

x-osa: metoda (SVM, zero-shot LLM, few-shot LLM),

y-osa: tačnost (Accuracy).

Vizuelizacija je namerno postavljena sa ylim(0, 1.1) da bi iznad stubića ostalo mesta za tekstualne anotacije.

Dodatno, petlja:

for index, row in results_df.iterrows():
    plt.text(index, row.Accuracy + 0.02, f"{row.Accuracy*100:.1f}%")


ispisuje procenat iznad svakog stubića (npr. 92.2%), čime grafikon postaje “samodovoljan” — posmatrač ne mora da pogađa numeričke vrednosti po visini stubića.

Suština ovog prikaza je da se na prvi pogled vidi: klasični ML pristup je značajno ispred LLM pristupa u ovom zadatku.

#4) Tumačenje razlika: zašto LLM pristupi zaostaju

Rezultati pokazuju da su zero-shot i few-shot performanse značajno niže od SVM baseline-a. Iako se intuitivno može očekivati da few-shot poboljša rezultate u odnosu na zero-shot, ovde se desilo suprotno (62% naspram 64%). To je realan scenario i često se dešava kada:

few-shot primeri nisu optimalno izabrani (premalo primera, domen nije dovoljno pokriven),

model previše “imitira” strukturu primera i postane konzervativniji,

prompt i JSON formatiranje uvedu dodatnu rigidnost koja dovodi do više “Neutral” fallback-ova.

U praksi, LLM pristupi su ovde najviše zakazivali na neutralnoj klasi: primećeno je da su neke stvarno pozitivne poruke završavale kao Neutral. To je tipična greška kada je sentiment blag (nema jasnih pozitivnih reči, više je “implicitno zadovoljstvo”), ili kada model, zbog pravila “ako nema jasnog sentimenta → Neutral”, postane previše oprezan i radije bira Neutral nego Positive.

#5) Konfuziona matrica za pobednički model (SVM)

Nakon globalnog poređenja metoda, kod prelazi na dublju analizu najboljeg modela:

Definišu se labels = ['Positive', 'Negative', 'Neutral'] kako bi redosled klasa bio konzistentan.

Računa se konfuziona matrica:

cm = confusion_matrix(y_val, y_pred, labels=labels)


Prikazuje se heatmap gde ose znače:

y-osa: stvarna klasa,

x-osa: predviđena klasa.

Ovakav prikaz omogućava da se vidi ne samo koliko je model tačan, već i koje greške dominiraju (npr. da li Positive često “beži” u Neutral, ili Negative u Neutral itd.). Pošto je SVM pobednik, ova matrica je praktično dokaz da model ne samo da ima visoku tačnost, već i da je greška raspoređena na prihvatljiv način.

#6) Zaključak ovog segmenta

Ovaj deo rada zatvara eksperiment na dva nivoa:

Makro nivo (bar-čart + tabela): jasno pokazuje da je trenirani TF-IDF + SVM model ubedljivo najbolji po accuracy (92,2%), dok LLM zero-shot i few-shot ostaju na ~64% i ~62%.

Mikro nivo (confusion matrix): pruža uvid u tipične greške najboljeg modela i služi kao osnova za interpretaciju.

Istovremeno, komentarisana slabost LLM pristupa (“guranje” pozitivnih u neutralnu) ukazuje na ključni problem: bez finog treniranja ili bolje kalibracije, LLM u ovom zadatku često igra na sigurno i bira Neutral, što direktno ruši recall za Positive klasu i obara ukupnu tačnost.